In [1]:
import numpy as np
import pandas as pd
import os,sys

In [2]:
path_data = os.path.join(os.getcwd(),os.pardir,os.pardir,'data')
data_list = os.listdir(path_data)
print(data_list)

['bleaching', 'fishing_activity', 'mmsi-daily-csvs-10-v2-2018', 'mmsi-daily-csvs-10-v2-2019']


In [3]:
year = 2018
filename = 'mmsi-daily-csvs-10-v2-' + str(year)
fishing_data_path = os.path.join(path_data,filename)
# encoding='ISO-8859-1' for read_csv

In [4]:
from datetime import datetime, timedelta
start_date = datetime(year, 1, 1)
end_date = datetime(year, 12, 31)
delta = timedelta(days=1)

aggregated_data = pd.DataFrame()
current_date = start_date
while current_date <= end_date:
    file_name = current_date.strftime('%Y-%m-%d') + '.csv'
    file_path = os.path.join(fishing_data_path, file_name)
    
    if os.path.exists(file_path):
        daily_data = pd.read_csv(file_path)
        daily_data = daily_data[['cell_ll_lat', 'cell_ll_lon', 'fishing_hours']]
        daily_data = daily_data[daily_data['fishing_hours'] > 0]
        
        # Filter for the specified latitude and longitude range
        # daily_data = daily_data[(daily_data['cell_ll_lat'] >= -20) & (daily_data['cell_ll_lat'] <= 40) &
        #                         (daily_data['cell_ll_lon'] >= 100) & (daily_data['cell_ll_lon'] <= 140)]
        
        # daily_data = daily_data[
        #     ((daily_data['cell_ll_lat'] >= -20) & (daily_data['cell_ll_lat'] <= 40) &
        #      (daily_data['cell_ll_lon'] >= 100) & (daily_data['cell_ll_lon'] <= 140)) |
        #     ((daily_data['cell_ll_lat'] >= 0) & (daily_data['cell_ll_lat'] <= 40) &
        #      (daily_data['cell_ll_lon'] >= -100) & (daily_data['cell_ll_lon'] <= -40))
        # ]
        
        # If the aggregated_data DataFrame is empty, initialize it with the first filtered DataFrame
        if aggregated_data.empty:
            aggregated_data = daily_data
        else:
            aggregated_data = pd.concat([aggregated_data, daily_data])

    current_date += delta

aggregated_data = aggregated_data.groupby(['cell_ll_lat', 'cell_ll_lon']).sum().reset_index()



In [ ]:
save_folder = os.path.join(os.getcwd(),os.pardir,os.pardir,'data','fishing_activity')
if not os.path.exists(save_folder):
    os.makedirs(save_folder)

save_file = os.path.join(save_folder,'fishing_activity_' + str(year) + '.csv')
aggregated_data.to_csv(save_file, index=False)